In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("final_project").getOrCreate()

In [2]:
spark

# load and explore data

In [223]:
df2004 = spark.read.csv("s3://jenova73/final/311_Service_Requests_for_2004.csv",
                      header = True, inferSchema= True)
df2005 = spark.read.csv("s3://jenova73/final/311_Service_Requests_for_2005.csv",
                      header = True, inferSchema= True)
df2006 = spark.read.csv("s3://jenova73/final/311_Service_Requests_for_2006.csv",
                      header = True, inferSchema= True)
df2007 = spark.read.csv("s3://jenova73/final/311_Service_Requests_for_2007.csv",
                      header = True, inferSchema= True)
df2008 = spark.read.csv("s3://jenova73/final/311_Service_Requests_for_2008.csv",
                      header = True, inferSchema= True)
df2009 = spark.read.csv("s3://jenova73/final/311_Service_Requests_for_2009.csv",
                      header = True, inferSchema= True)
df2010 = spark.read.csv("s3://jenova73/final/311_Service_Requests_from_2010_to_Present (1).csv",
                      header = True, inferSchema= True)





In [98]:
data = spark.read.csv("s3://jenova73/final/311_Service_Requests_for_2004.csv",
                      header = True, inferSchema= True)
                    

In [21]:
data.show(2)

+----------+--------------------+--------------------+------+--------------------+--------------------+--------------------+-------------+------------+----------------+-----------+--------------+--------------+---------------------+---------------------+------------+----+--------+-------------+------+--------+------------------------------+---------------+-----------+--------------------------+--------------------------+------------------+------------+-----------+-------------+-------------+-----------+-------------------+--------------+-----------+------------+-----------+----------------+----------------------------+------------+--------------------+---------------------+-------------------+------------------------+---------+----------------------+---------------+---------------+-------------------+--------+---------+--------+
|Unique Key|        Created Date|         Closed Date|Agency|         Agency Name|      Complaint Type|          Descriptor|Location Type|Incident Zip|Incident

In [8]:
data.printSchema()

root
 |-- Unique Key: integer (nullable = true)
 |-- Created Date: string (nullable = true)
 |-- Closed Date: string (nullable = true)
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- Incident Address: string (nullable = true)
 |-- Street Name: string (nullable = true)
 |-- Cross Street 1: string (nullable = true)
 |-- Cross Street 2: string (nullable = true)
 |-- Intersection Street 1: string (nullable = true)
 |-- Intersection Street 2: string (nullable = true)
 |-- Address Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Landmark: string (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Due Date: string (nullable = true)
 |-- Resolution Action Updated Date: string (nullable = true)
 |-- Communit

In [22]:
data.count()

1118286

In [177]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import to_timestamp

In [28]:
data.dropna()

DataFrame[Unique Key: int, Created Date: string, Closed Date: string, Agency: string, Agency Name: string, Complaint Type: string, Descriptor: string, Location Type: string, Incident Zip: string, Incident Address: string, Street Name: string, Cross Street 1: string, Cross Street 2: string, Intersection Street 1: string, Intersection Street 2: string, Address Type: string, City: string, Landmark: string, Facility Type: string, Status: string, Due Date: string, Resolution Action Updated Date: string, Community Board: string, Borough: string, X Coordinate (State Plane): int, Y Coordinate (State Plane): int, Park Facility Name: string, Park Borough: string, School Name: string, School Number: string, School Region: string, School Code: string, School Phone Number: string, School Address: string, School City: string, School State: string, School Zip: string, School Not Found: string, School or Citywide Complaint: string, Vehicle Type: string, Taxi Company Borough: string, Taxi Pick Up Locat

In [29]:
data.count()

1118286

### Data Cleaning

In [45]:
columns = ['Created Date','Closed Date','Agency','Agency Name',
          'Complaint Type','Descriptor','Location Type','Incident Zip',
          'City','Status','Community Board','Borough','Park Borough',
          'Latitude','Longitude']

In [99]:
data=data.select(*columns)

In [206]:
from pyspark.sql.functions import to_date, to_timestamp, datediff,year, month
df1=data.withColumn("start_date", to_date('Created Date','MM/dd/yyyy HH:mm:ss'))
df1=df1.withColumn("end_date", to_date('Closed Date','MM/dd/yyyy HH:mm:ss'))
df1=df1.withColumn('Time',datediff('end_date','start_date'))
df1= df1.withColumn('Timebin',F.when(df1.Time<3, 1).otherwise(F.when(df1.Time<6,2).otherwise(F.when(df1.Time<11,3).otherwise(4))))
df1=df1.withColumn('Year',year('start_date'))
df1=df1.withColumn('Month',month('start_date'))
df1=df1.drop('Created Date','Closed Date','start_date','end_date')


In [207]:
df1.printSchema()

root
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Community Board: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Park Borough: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Timebin: integer (nullable = false)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)



# prepare data

In [213]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model
import pandas as pd
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import col

In [214]:
(train, test) = (df1.randomSplit([0.75,0.25]))

### change string or integer into index

In [216]:
df1.printSchema()

root
 |-- Agency: string (nullable = true)
 |-- Agency Name: string (nullable = true)
 |-- Complaint Type: string (nullable = true)
 |-- Descriptor: string (nullable = true)
 |-- Location Type: string (nullable = true)
 |-- Incident Zip: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Community Board: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Park Borough: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Time: integer (nullable = true)
 |-- Timebin: integer (nullable = false)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)



In [ ]:
vendor_indexer = StringIndexer(inputCol = 'vendor_id',outputCol='vendor_index')
vendor_indexer = StringIndexer(inputCol = 'vendor_id',outputCol='vendor_index')
vendor_indexer = StringIndexer(inputCol = 'vendor_id',outputCol='vendor_index')
